In [1]:
! pip install pandas openpyxl sentence-transformers scikit-learn matplotlib seaborn

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util

# 1. 환경 설정 및 모델 로드
plt.rcParams['font.family'] = 'Malgun Gothic' # Windows 기준
plt.rcParams['axes.unicode_minus'] = False

# KoE5 모델 로드
model = SentenceTransformer('intfloat/multilingual-e5-small')

# 최상위 경로 설정 (모든 지역구가 포함된 폴더)
root_path = r'G:\내 드라이브\빅데이터_부산\텍스트(네이버 리뷰)'

# 2. 불용어 사전 정의 (분석 목적에 맞게 추가 가능)
# 관광지 리뷰에서 변별력을 떨어뜨리는 단어들을 제거합니다.
stopwords = # [전체 채팅 분석 기반] 부산 관광지 유사도 분석용 최종 통합 불용어 리스트
ST_STOPWORDS = [
    # 1. 지역 상위어 (부산 내 장소 비교 시 변별력을 완전히 파괴하는 주범)
    '부산', '부산시', '부산여행', '부산가볼만한곳', '부산항', '부경', '경남', '부산역',
    
    # 2. 타 지역 지명 오염 (구덕야영장, 서동미로시장 등에서 발견된 오염 데이터 차단)
    '경기도', '수원', '인천', '부여', '원주', '방콕', '제주', '강원도', '망원', '잠원', '서울', '경기',
    
    # 3. 비유적 표현 및 중의성 단어 (아미르공원 '한강' 비유 등 분석 반영)
    '한강', '한국', '세계', '국내', '다른', '바로', '진짜', '어디', '현지', '이후',
   
    # 5. 일반적인 관광 공통어 (모든 블로그 리뷰에 등장하여 유사도를 높게 만드는 단어)
    '여행', '사진', '사진가', '모습', '생각', '정도', '오늘', '이번', '추천', '인기', '방문', 
    '정보', '위치', '문화', '건물', '관련', '보고', '정말', '준비', '시작', '다양한', '함께',
    
    # 6. 수치/행위 및 단위 (데이터 정교화 과정에서 발견)
    '인분', '주문', '메뉴', '가격', '체험', '관람', '이용', '내용', '상황', '확인'
]

def preprocess_text(text):
    if pd.isna(text): return ""
    # 특수문자 제거
    text = re.sub(r'[^가-힣0-9\s]', ' ', str(text))
    # 불용어 제거
    words = text.split()
    words = [w for w in words if w not in stopwords and len(w) > 1]
    text = " ".join(words)
    # E5 모델 접두사 추가
    return f"passage: {text}"

# 3. 모든 폴더 순회 및 데이터 수집
place_names = []
place_vectors = []

print("전체 지역구 데이터 통합 분석 시작...")

# root_path 내의 모든 하위 폴더(지역구) 탐색
for addr, dirs, files in os.walk(root_path):
    for file_name in files:
        if file_name.endswith(('.csv', '.xlsx')):
            file_path = os.path.join(addr, file_name)
            
            # 지역구 이름 파악 (폴더명)
            district_name = os.path.basename(addr)
            # 관광지 이름 (파일명)
            place_name = f"{district_name}_{os.path.splitext(file_name)[0]}"
            
            try:
                # 데이터 로드
                if file_name.endswith('.csv'):
                    df = pd.read_csv(file_path)
                else:
                    df = pd.read_excel(file_path)
                
                # '제목'과 '본문' 컬럼 존재 여부 확인
                if '제목' not in df.columns or '본문' not in df.columns:
                    continue
                
                # 텍스트 결합 및 전처리
                texts = (df['제목'].fillna('') + " " + df['본문'].fillna('')).apply(preprocess_text).tolist()
                
                # 너무 짧은 리뷰 제외
                valid_texts = [t for t in texts if len(t) > 20]
                
                if not valid_texts:
                    continue

                # 각 리뷰별 임베딩 후 평균(Mean Pooling)
                with torch.no_grad():
                    embeddings = model.encode(valid_texts, convert_to_tensor=True)
                    avg_vector = torch.mean(embeddings, dim=0)
                
                place_names.append(place_name)
                place_vectors.append(avg_vector)
                print(f"  - [성공] {place_name} ({len(valid_texts)}건)")
                
            except Exception as e:
                print(f"  - [오류] {place_name}: {e}")

# 4. 유사도 계산
if len(place_vectors) < 2:
    print("분석 가능한 관광지 데이터가 부족합니다.")
else:
    all_embeddings = torch.stack(place_vectors)
    sim_matrix = util.cos_sim(all_embeddings, all_embeddings).cpu().numpy()

    # 5. 결과 시각화 (데이터가 많을 경우 상위 일부만 그리거나 파일로 저장)
    # 전체를 다 그리면 너무 크므로 히트맵은 파일로 고화질 저장
    plt.figure(figsize=(20, 18))
    sns.heatmap(sim_matrix, cmap='YlGnBu', xticklabels=place_names, yticklabels=place_names)
    plt.title('부산 전체 관광지 문맥 유사도 매트릭스')
    plt.tight_layout()
    plt.savefig('busan_all_similarity.png', dpi=300)
    print("\n[시각화 완료] 'busan_all_similarity.png' 파일이 생성되었습니다.")

    # 6. 지역구 간 경계를 넘나드는 유사 관광지 추천 (결과 분석)
    print("\n" + "="*60)
    print("의외의 유사성: 다른 지역구지만 분위기가 비슷한 곳 TOP 3")
    print("="*60)

    for i, name in enumerate(place_names):
        scores = sim_matrix[i]
        sorted_idx = np.argsort(scores)[::-1]
        
        current_district = name.split('_')[0]
        count = 0
        print(f"\n[{name}]와 분위기가 유사한 타 지역 명소:")
        
        for idx in sorted_idx:
            target_name = place_names[idx]
            target_district = target_name.split('_')[0]
            
            # 본인 제외 및 타 지역구만 필터링
            if name != target_name and current_district != target_district:
                print(f"  - {target_name} (유사도: {scores[idx]:.4f})")
                count += 1